# 網頁資料擷取實務
- sqlite教學
- 爬蟲資料與資料庫存取、互動選單設計
- 中文斷字

## sqlite 相關應用

In [ ]:
#編輯sqlite檔案  輸入

import sqlite3
conn = sqlite3.connect('___.sqlite')
conn.execute('insert into student values(1,"XXX");')
conn.commit()
conn.close()


In [ ]:
#編輯sqlite檔案  輸出
import sqlite3
conn = sqlite3.connect('scores.sqlite')
cursor = conn.execute('SELECT * FROM student;')

#cursor是甚麼? 怎麼看?

#for row in cursor:
#    print('No {}:{}'.format(row[0],row[1]))
#conn.close()


In [ ]:
### 資料庫應用程式，輸入輸出刪除修改

import sqlite3

conn = sqlite3.connect('___.sqlite')

def disp_menu():
    print("學生資料編輯")
    print("------------")
    print("1.新增")
    print("2.編輯")
    print("3.刪除")
    print("4.顯示所有學生")
    print("0.結束")
    print("------------")

def append_data():
    while True:
        no = int(input("請輸入學生座號(-1停止輸入):"))
        if no == -1: break
        name = input("請輸入學生姓名:")
        sqlstr = "select * from student where stdno={};".format(no)
        cursor = conn.execute(sqlstr)
        if len(cursor.fetchall()) > 0:
            print("您輸入的座號已經有資料了")
        else:
            sqlstr = "insert into student values({},'{}');".format(no,name)
            conn.execute(sqlstr)
            conn.commit()


def edit_data():
    no = input("請輸入要編輯的學生座號:")
    sqlstr = "select * from student where stdno={};".format(no)
    cursor = conn.execute(sqlstr)
    rows = cursor.fetchall()
    if len(rows) > 0:
        print("目前的學生姓名:",rows[0][1])
        name = input("請輸入學生姓名：")
        sqlstr = "update student set name='{}' where stdno={};".format(name, no)
        conn.execute(sqlstr)
        conn.commit()
    else:
        print("找不到要編輯的學生座號")

def del_data():
    no = input("請輸入要刪除的學生座號:")
    sqlstr = "select * from student where stdno={};".format(no)
    cursor = conn.execute(sqlstr)
    rows = cursor.fetchall()
    if len(rows) > 0:
        print("你目前要刪除的是座號{}的{}".format(rows[0][0], rows[0][1]))
        answer = input("確定要刪除嗎？(y/n)")
        if answer == 'y' or answer == 'Y':
            sqlstr = "delete from student where stdno={};".format(no)
            conn.execute(sqlstr)
            conn.commit()
            print("已刪除指定的學生...")
    else:
        print("找不到要刪除的學生")

def disp_data():
    cursor = conn.execute('select * from student;')
    for row in cursor:
        print("No {}: {}".format(row[0],row[1]))



while True:
    disp_menu()
    choice = int(input("請輸入您的選擇:"))
    if choice == 0 : break
    if choice == 1: 
        append_data()
    elif choice == 2:
        edit_data()
    elif choice == 3:
        del_data()
    elif choice == 4:
        disp_data()
    else: break
    x = input("請按Enter鍵回主選單")


##  網頁資料爬好存到資料庫

In [ ]:
# 把網頁資料爬好存到資料庫
# 程式 10-1.py (Python 3 version)

import sqlite3
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as pt

conn = sqlite3.connect('gasoline.sqlite')

#選單
def disp_menu():
    print("中油歷年油價查詢系統")
    print("------------")
    print("1.從網站載入最新油價")
    print("2.顯示歷年油價資訊")
    print("3.最近10週油價資訊")
    print("4.油價走勢圖")
    print("0.結束")
    print("------------")


#新增(爬)資料
def fetch_data():
    url = 'http://new.cpc.com.tw/division/mb/oil-more4.aspx'

    html = requests.get(url).text
    sp = BeautifulSoup(html, 'html.parser')
    data = sp.find_all('span', {'id':'Showtd'})
    rows = data[0].find_all('tr')

    prices = list()
    for row in rows:
        cols = row.find_all('td')
        if len(cols[1].text) > 0:
            item = [cols[0].text, cols[1].text, \
                    cols[2].text, cols[3].text]
            prices.append(item)
    for p in prices:
        sqlstr = "select * from prices where gdate='{}';".format(p[0])
        cursor = conn.execute(sqlstr)
        if len(cursor.fetchall()) == 0:
            g92 = 0 if p[1]=='' else float(p[1])
            g95 = 0 if p[2]=='' else float(p[2])
            g98 = 0 if p[3]=='' else float(p[3])
            sqlstr = "insert into prices values('{}', {}, {}, {});". \
                format(p[0], g92, g95, g98)
            print(sqlstr)
            conn.execute(sqlstr)
            conn.commit()

#展示
def disp_10data():
    cursor = conn.execute('select * from prices order by gdate desc;')
    n = 0
    for row in cursor:
        print("日期：{}，92無鉛：{}，95無鉛：{}，98無鉛：{}". \
            format(row[0],row[1],row[2],row[3]))
        n = n + 1
        if n == 10:
            break

            
#畫圖表
def chart():
    data = []
    cursor = conn.execute('select * from prices order by gdate;')
    for row in cursor:
        data.append(list(row))
    x = np.arange(0,len(data))
    dataset = [list(), list(), list()]
    for i in range(0, len(data)):
        for j in range(0,3):
            dataset[j].append(data[i][j+1])
    w = np.array(dataset[0])
    y = np.array(dataset[1])
    z = np.array(dataset[2])
    pt.ylabel("NTD$")
    pt.xlabel("Weeks ( {} --- {} )".format(data[0][0], data[len(data)-1][0]))
    pt.plot(x, w, color="blue", label="92")
    pt.plot(x, y, color="red", label="95")
    pt.plot(x, z, color="green", label="98")
    pt.xlim(0,len(data))
    pt.ylim(10,40)
    pt.title("Gasoline Prices Trend (Taiwan)")
    pt.legend()
    pt.show()

    
#SHOW資料與選項
def disp_alldata():
    cursor = conn.execute('select * from prices order by gdate desc;')
    n = 0
    for row in cursor:
        print("日期：{}，92無鉛：{}，95無鉛：{}，98無鉛：{}". \
            format(row[0],row[1],row[2],row[3]))
        n = n + 1
        if n == 20:
            x = input("請按Enter鍵繼續...(Q:回主選單)")
            if x == 'Q' or x == 'q': break
            n = 0



while True:
    disp_menu()
    choice = int(input("請輸入您的選擇:"))
    if choice == 0 : break
    if choice == 1: 
        fetch_data()
    elif choice == 2:
        disp_alldata()
    elif choice == 3:
        disp_10data()
    elif choice == 4:
        chart()
    else: break
    x = input("請按Enter鍵回主選單")


## 中文斷詞

In [ ]:
# _*_ coding: utf-8 _*_
# 蒐集新聞&中文

import requests
from bs4 import BeautifulSoup

url = 'http://www.appledaily.com.tw/appledaily/hotdaily/headline'

news_page = requests.get(url)
news = BeautifulSoup(news_page.text, 'html.parser')

news_title = news.find_all('div', {'class': 'aht_title'})

news_title[0].find_all('a')[0].text

#headlines = ''
#for t in news_title:
#    title = t.find_all('a')[0]
#    headlines += title.text

#print(headlines)



In [ ]:
#jieba功能說明
#pip install jieba 安裝 自訂字典

import jieba
#jieba.load_userdict("userdict.txt")
words = jieba.cut("他來到台北市臺北醫學大學上鈞哥的Python程式設計課程，老師非常感動，有滿滿的大平台")


for i in words:
    print(i)

其他jieba功能請參考
https://github.com/fxsjy/jieba


In [ ]:
# _*_ coding: utf-8 _*_
# 中文段詞&排序

import requests, jieba, operator
from bs4 import BeautifulSoup

url = 'http://www.appledaily.com.tw/appledaily/hotdaily/headline'

news_page = requests.get(url)
news = BeautifulSoup(news_page.text, 'html.parser')

news_title = news.find_all('div', {'class': 'aht_title'})

headlines = ''
for t in news_title:
    title = t.find_all('a')[0]
    headlines += title.text

words = jieba.cut(headlines)

word_count = dict()

for word in words:
    if word in word_count.keys():
        word_count[word] += 1
    else:
        word_count[word] = 1

    sorted_wc = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)

for item in sorted_wc:
    if item[1]>0:
        print(item)
    else:
        break

# 作業
- 尋找一篇網路的文章斷字
- 利用jieba斷字與自定義字典功能
- 按照字詞出現平率由大到小排列
- 需移除標點符號

''':!),.:;?]}¢'"、。〉》」』】〕】〞︰︱︳﹐､﹒ ﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠ 々∥•‧ˇˉ─--′』」([{£¥'"‵〈《「『【〔【（［｛￡￥〝︵︷︹︻ ︽︿﹁﹃﹙﹛﹝（｛「『-—_…'''



In [ ]:
# 單篇新聞範例參考(未完成)

import requests
from bs4 import BeautifulSoup

# 自創字典檔
jieba.load_userdict("userdict2.txt")

url = 'https://anntw.com/articles/20161211-Y8vN'

news_page = requests.get(url)
news = BeautifulSoup(news_page.text, 'html.parser')
contentAll = news.find_all("div",{"class":"markdown-body"})

content = ''
for t in contentAll:
    content += t.text
content